In [ ]:
%%bash
# Set environment variables
export MODEL_NAME="openai/gpt-oss-20b"
export LOCAL_MODEL_PATH="./downloaded_models/gpt-oss-20b"

# Download the model using huggingface-cli
huggingface-cli download ${MODEL_NAME} \
    --local-dir ${LOCAL_MODEL_PATH} \
    --local-dir-use-symlinks False

In [1]:
import os
import time
from huggingface_hub import HfApi
from nemo_microservices import NeMoMicroservices

/home/shadeform/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Set environment variables - Update these to match your deployment
NAMESPACE = "my-org"
MODEL_NAME = "openai-gpt-oss-20b"
MODEL_VERSION = os.getenv("MODEL_VERSION", f"{int(time.time())}")  # Unique version for this run
NEMO_BASE_URL = os.getenv("NEMO_BASE_URL", "http://nemo.test")
DATASTORE_URL = os.getenv("DATASTORE_URL", "http://data-store.test")
NIM_BASE_URL = os.getenv("NIM_URL", "http://nim.test")

In [11]:
hf_api = HfApi(endpoint=f"{DATASTORE_URL}/v1/hf", token="")

In [12]:
client = NeMoMicroservices(
    base_url=NEMO_BASE_URL,
    inference_base_url=NIM_BASE_URL,
)

# Create namespace
namespace = client.namespaces.create(id=NAMESPACE)
print(f"Created namespace: {namespace.id}")

ConflictError: Error code: 409 - {'detail': 'Namespace already exists'}

In [13]:
# Create model repository in datastore
repo_id = f"{NAMESPACE}/{MODEL_NAME}"
hf_api.create_repo(repo_id, repo_type="model", exist_ok=True)
model_info = hf_api.model_info(repo_id)
print(f"Created model repository: {repo_id}")

Created model repository: my-org/openai-gpt-oss-20b


In [ ]:
# Initialize the HF API client for the internal datastore
# Note: This assumes DATASTORE_URL is already defined from previous code block
from huggingface_hub import HfApi
hf_api = HfApi(endpoint=f"{DATASTORE_URL}/v1/hf", token="")

# Upload the entire model directory to datastore
LOCAL_MODEL_PATH = "/home/shadeform/downloaded_models/gpt-oss-20b"
repo_id = f"{NAMESPACE}/{MODEL_NAME}"

# Upload all files in the model directory
hf_api.upload_folder(
    repo_id=repo_id,
    folder_path=LOCAL_MODEL_PATH,
    repo_type="model",
    revision="main",
    commit_message=f"Upload {MODEL_NAME} model files"
)

print(f"Model files uploaded to {repo_id}")

RuntimeError: Error while uploading 'model-00000-of-00002.safetensors' to the Hub.

In [ ]:
# Create model entity in Entity Store
model = client.models.create(
    name=MODEL_NAME,
    namespace=NAMESPACE,
    description=f"Private {MODEL_NAME} model imported for customization",
    artifact={
        "files_url": f"hf://models/{repo_id}",
        "backend_engine": "hugging_face",
        "status": "upload_completed",
    },
    spec={
        "num_parameters": 20_000_000_000,
        "context_size": 4096,
        "is_chat": True,
        "num_virtual_tokens": -1,
    },
    peft={
        "finetuning_type": "all_weights",
    }
)
print(f"Created model entity: {model.namespace}/{model.name}")